In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys

sys.path.append("../")

In [2]:
from __future__ import annotations

import os
import types
import datetime

import pyresample.kd_tree
import pyproj
import pandas as pd
import numpy as np

import xarray as xr


from src.mesoformer.typing import Self, Concatenate, TypeVarTuple, Unpack, NestedSequence, Any
from src.mesoformer.generic import Array
from src.mesoformer.utils import normalized_scale, sort_unique
from src.mesoformer.datasets.core import Mesoscale
from src.mesoformer.datasets.metadata import ERA5Enum, URMAEnum, CFDatasetEnum

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
urma_dvars = list(URMAEnum)

era5_store = os.path.join(_test_data, "era5.zarr")
era5_dvars = list(ERA5Enum)

print(era5_dvars, urma_dvars, sep="\n")

[ERA5Enum['geopotential'], ERA5Enum['specific_humidity'], ERA5Enum['temperature'], ERA5Enum['u_component_of_wind'], ERA5Enum['v_component_of_wind'], ERA5Enum['vertical_velocity']]
[URMAEnum['total_cloud_cover'], URMAEnum['ceiling'], URMAEnum['u_wind_component_10m'], URMAEnum['v_wind_component_10m'], URMAEnum['wind_speed_10m'], URMAEnum['wind_speed_gust'], URMAEnum['wind_direction_10m'], URMAEnum['temperature_2m'], URMAEnum['dewpoint_temperature_2m'], URMAEnum['specific_humidity_2m'], URMAEnum['surface_pressure'], URMAEnum['visibility'], URMAEnum['orography']]


In [3]:
from src.mesoformer.generic import StrEnum, EnumMetaBase


# class OrderedDims(StrEnum, metaclass=EnumMetaBase):
#     TIME = "t"
#     LEVEL = "z"
#     LATITUDE = "y"
#     LONGITUDE = "x"


# latitude
# tuple(OrderedDims)

In [4]:
_URMA_DATASET = xr.open_zarr(urma_store)
_URMA_DATASET

<xarray.Dataset>
Dimensions:                  (time: 2, y: 1597, x: 2345)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-02 2019-01-02T01:0...
Dimensions without coordinates: y, x
Data variables: (12/14)
    ceiling                  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    latitude                 (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    longitude                (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    ...                       ...
    u_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    visibility               (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [5]:
if not os.path.exists(era5_store):
    _google_store = "gs://weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2"
    xr.open_zarr(_google_store)[era5_dvars].sel(time=_URMA_DATASET.time).to_zarr(era5_store, mode="w")

_ERA5_DATASET = xr.open_zarr(era5_store)
_ERA5_DATASET

<xarray.Dataset>
Dimensions:              (time: 2, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Data variables:
    geopotential         (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [6]:
def get_urma() -> xr.Dataset:
    return _URMA_DATASET


def get_era5() -> xr.Dataset:
    return _ERA5_DATASET

In [7]:
from src.mesoformer.typing import StrPath
from src.mesoformer.typing import Sequence
from src.mesoformer.typing import Sequence, DictStrAny
from src.mesoformer.generic import Data
from src.mesoformer.datasets.metadata import MetadataMixin, X, Y, Z, T
from typing import Mapping, Hashable
from typing import Iterable

VariableLike = type[CFDatasetEnum] | CFDatasetEnum | Sequence[CFDatasetEnum]


class CFDataset(MetadataMixin):
    def __init__(self, ds: xr.Dataset, dvars: VariableLike) -> None:
        super().__init__()
        self._enum, self._dvars = _, dvars = self._validate_variables(dvars)
        ds = ds.swap_dims(self.dims.map(ds.dims)).set_coords(self.get_coords())[dvars]

        if kwargs := self._get_ordered_dims(ds.dims):
            ds = ds.expand_dims(**kwargs)

        if tuple(ds.dims) != tuple(self.dims):
            raise ValueError(f"Dataset dimensions {ds.dims} do not match {self.dims}")

        self._ds = ds = ds

    @classmethod
    def from_zarr(cls, path: StrPath, dvars: VariableLike) -> CFDataset:
        return cls(xr.open_zarr(path), dvars)

    @staticmethod
    def _validate_variables(dvars: VariableLike) -> tuple[type[CFDatasetEnum], list[CFDatasetEnum]]:
        if isinstance(dvars, type):
            assert issubclass(dvars, CFDatasetEnum)
            enum = dvars
            dvars = list(dvars)
        elif isinstance(dvars, CFDatasetEnum):
            enum = dvars.__class__
            dvars = [dvars]
        else:
            enum = dvars[0].__class__
            dvars = list(dvars)

        for dvar in dvars:
            assert isinstance(dvar, enum)
        return enum, dvars

    def _get_ordered_dims(self, dims: Iterable[Hashable]) -> DictStrAny:
        if diff := list(self.dims.difference(dims)):
            return dict(dim=diff, axis=[self.dims.order.index(member) for member in diff])
        return {}

    @property
    def metadata(self):
        return self._enum.md

    @property
    def enum(self) -> type[CFDatasetEnum]:
        return self._enum

    @property
    def dvars(self) -> list[CFDatasetEnum]:
        return self._dvars

    @property
    def ds(self) -> xr.Dataset:
        return self._ds

    def __repr__(self) -> str:
        return self.ds.__repr__()

    def _repr_html_(self) -> str:
        return self.ds._repr_html_()

    def to_array(self):
        return self.ds.to_array().transpose(X, Y, ...)

    @property
    def x(self) -> xr.DataArray:
        return self.ds[X]

    @property
    def y(self) -> xr.DataArray:
        return self.ds[Y]

    @property
    def z(self) -> xr.DataArray:
        return self.ds[Z]

    @property
    def t(self) -> xr.DataArray:
        return self.ds[T]

    @property
    def latitude(self) -> xr.DataArray:
        return self.ds["latitude"]


cf = CFDataset(get_urma(), URMAEnum("ceil", "vis"))
print(cf.x, cf.y, cf.z, cf.t, cf.latitude, sep="\n======================\n")

<xarray.DataArray OrderedDims['x'] (x: 2345)>
array([   0,    1,    2, ..., 2342, 2343, 2344])
Dimensions without coordinates: x
<xarray.DataArray OrderedDims['y'] (y: 1597)>
array([   0,    1,    2, ..., 1594, 1595, 1596])
Dimensions without coordinates: y
<xarray.DataArray OrderedDims['z'] (z: 1)>
array([0])
Dimensions without coordinates: z
<xarray.DataArray OrderedDims['t'] (t: 2)>
array([0, 1])
Coordinates:
    time     (t) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: t
<xarray.DataArray 'latitude' (y: 1597, x: 2345)>
dask.array<open_dataset-latitude, shape=(1597, 2345), dtype=float64, chunksize=(1597, 2345), chunktype=numpy.ndarray>
Coordinates:
    latitude   (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    longitude  (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
Dimensions without coordinates: y, x
Attributes:
    long_name:      latitude
    standard_name:  latitude
    units:          degrees_north


In [8]:
cf = CFDataset(get_era5(), ERA5Enum)
print(cf.x, cf.y, cf.z, cf.t, sep="\n===\n")

<xarray.DataArray OrderedDims['x'] (x: 1440)>
array([   0,    1,    2, ..., 1437, 1438, 1439])
Coordinates:
    longitude  (x) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Dimensions without coordinates: x
===
<xarray.DataArray OrderedDims['y'] (y: 721)>
array([  0,   1,   2, ..., 718, 719, 720])
Coordinates:
    latitude  (y) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
Dimensions without coordinates: y
===
<xarray.DataArray OrderedDims['z'] (z: 37)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36])
Coordinates:
    level    (z) int64 1 2 3 5 7 10 20 30 ... 825 850 875 900 925 950 975 1000
Dimensions without coordinates: z
===
<xarray.DataArray OrderedDims['t'] (t: 2)>
array([0, 1])
Coordinates:
    time     (t) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: t


In [9]:
import matplotlib.pyplot as plt


class Dataset:
    """
    # CONUS and Northern Hemisphere Grids

    https://graphical.weather.gov/docs/ndfdSRS.htm#:~:text=The%20NDFD%20uses%20the%20World%20Geodetic%20System,1984%20%28WGS84%29%20ellipsoid%20for%20its%20horizontal%20datum.

    Grid Parameter	    CONUS 2.5km
    Number of Points	2953665
    Projection Type	    Lambert Conformal
    Shape of Earth      Sphere
    Earth Radius	    6371.2 km
    Number of Points on the parallel	2145
    Number of Points on the Meridian	1377
    Latitude1:	20.191999
    Longitude1:	238.445999
    u/v vectors relative to:	easterly/northerly
    Dx	2539.703 m
    Dy	2539.703 m
    GRIB2 grid, scan mode	64 (0100)
    Scan i/x direction	positive
    Scan j/y direction	positive
    Consecutive points in	i/x direction
    Adjacent rows scan in	same direction
    Mesh Latitude	25
    Orientation Longitude	265
    Which Pole is on the Plane	north
    Is Projection Bi-polar	no
    Tangent Latitude1	25
    Tangent Latitude2	25
    Southern Latitude	-90
    Southern Longitude	0
    """

    cf = types.MappingProxyType(
        {
            "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
            "projected_crs_name": "NDFD",
            "semi_major_axis": 6378137.0,
            "semi_minor_axis": 6356752.31424518,
            "inverse_flattening": 298.25722356301,
            "reference_ellipsoid_name": "WGS 84",
            "longitude_of_prime_meridian": 0.0,
            "prime_meridian_name": "Greenwich",
            "horizontal_datum_name": "WGS84",
            "latitude_of_projection_origin": 20.191999,
            "longitude_of_projection_origin": 238.445999,
            "false_easting": 0.0,
            "false_northing": 0.0,
        }
    )

    def __init__(self, ds: xr.Dataset):
        self.da = da = ds.to_array().transpose("x", "y", ...)
        self.lons = lons = (da["longitude"].to_numpy() + 180) % 360 - 180
        self.lats = lats = da["latitude"].to_numpy()

        self._source_definition = pyresample.geometry.AreaDefinition(
            self.cf["geographic_crs_name"],
            "National Digital Forecast Database Grid",
            self.cf["projected_crs_name"],
            self.get_crs("lambert_conformal_conic", standard_parallel=25),
            da["y"].size,
            da["x"].size,
            area_extent=self.area_extent,
            lons=lons,
            lats=lats,
        )

    def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
        return self._source_definition

    def get_target_definition(
        self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
    ) -> pyresample.geometry.AreaDefinition:
        crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
        return pyresample.geometry.AreaDefinition(
            "target_projection",
            "description",
            None,
            crs,
            width=width,
            height=height,
            area_extent=area_extent,
        )

    def get_crs(
        self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
    ) -> pyproj.CRS:
        origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
        if latitude is not None:
            origin["latitude_of_projection_origin"] = latitude

        if longitude is not None:
            origin["longitude_of_projection_origin"] = longitude

        return pyproj.CRS.from_cf(self.cf | origin | kwargs)

    @property
    def area_extent(self) -> list[float]:
        return [
            self.lons.min(),
            self.lats.min(),
            self.lons.max(),
            self.lats.max(),
        ]

    def resample_on_center(
        self,
        longitude: float,
        latitude: float,
        *,
        width=256,
        height=256,
        dx=100,
        dy=100,
        scale_x=1,
        scale_y=1,
        units="km",
    ):
        if units == "km":
            dx *= 1000
            dy *= 1000

        dx /= 2
        dy /= 2

        height *= scale_y
        width *= scale_x

        source = self.get_source_definition()
        data = self.da.to_numpy()

        area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
        target = self.get_target_definition(latitude, longitude, width, height, area_extent)

        return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


dataset = Dataset(get_urma().isel(time=0))
data = dataset.resample_on_center(longitude=-89.835, latitude=38.54)
H, W, C = data.shape

fig, axes = plt.subplots(1, C, figsize=(20, 5))

for i in range(C):
    ax = axes[i]
    ax.imshow(data[:, :, i], origin="upper", cmap="terrain")
    ax.set_xticks([])
    ax.set_yticks([])

KeyError: 'longitude'